# Time Series Anomaly Detection Exercises

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mason_functions as mf
import warnings
warnings.filterwarnings('ignore')

import env
import os

Exercises
file name: time_series_anomaly_detection.py or time_series_anomaly_detection.ipynb
The dataset for these exercises lives in the Codeup Data Science MySQL Server. The database name is curriculum_logs.
Hint: You will need to explore the database and significantly adjust your acquisition step (yay SQL!)

In [2]:
# define url 
url = mf.get_db_url('curriculum_logs')

# define sql query
sql = '''
SELECT * 
FROM logs
LEFT JOIN cohorts on logs.cohort_id = cohorts.id
'''

# set up if-conditional to see if a .csv is cached
if os.path.isfile('curriculum_logs_unfiltered.csv'):
    af = pd.read_csv('curriculum_logs_unfiltered.csv', index_col = 0)
    
else:

    # use pandas's read_sql function to retrieve data from codeup db
    af = pd.read_sql(sql, url)
    
    # cache data
    af.to_csv('curriculum_logs_unfiltered.csv')

# preview query results
af.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0


In [3]:
# get df info
af.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          847330 non-null  float64
 7   name        847330 non-null  object 
 8   slack       847330 non-null  object 
 9   start_date  847330 non-null  object 
 10  end_date    847330 non-null  object 
 11  created_at  847330 non-null  object 
 12  updated_at  847330 non-null  object 
 13  deleted_at  0 non-null       float64
 14  program_id  847330 non-null  float64
dtypes: float64(4), int64(1), object(10)
memory usage: 109.9+ MB


In [4]:
# set a list of columns I don need
don_need = ['id', 'slack', 'deleted_at']

# drop these columns
af = af.drop(columns = don_need)

In [5]:
# confirm changes with info
af.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   name        847330 non-null  object 
 7   start_date  847330 non-null  object 
 8   end_date    847330 non-null  object 
 9   created_at  847330 non-null  object 
 10  updated_at  847330 non-null  object 
 11  program_id  847330 non-null  float64
dtypes: float64(2), int64(1), object(9)
memory usage: 89.3+ MB


In [6]:
# preview final base frame
af.head()

,date,time,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0


In [7]:
# cache data in a .csv
af.to_csv('anomaly_frame.csv')

In [8]:
#concatenate date and time for swag
af['date_time'] = af.date + ' ' + af.time

# set date to datetime object using pandas
af.date_time = pd.to_datetime(af.date_time, format = '%Y-%m-%d %H:%M:%S')

# set index to datetime object created
af = af.set_index('date_time')

#drop originals
af = af.drop(columns = ['date', 'time'])

In [9]:
# verify
af.head(1)

,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id
date_time,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0


Actually, let's just set all of the dates/ times to datetime objects. Why not?

In [10]:
# convert date/ time columns to datetime objects
# af.time = pd.Timestamp(af.time) # actually scratch that for now that is hard why dont i just concatenate the date and the time and then format it hmmm
af.start_date = pd.to_datetime(af.start_date)
af.end_date = pd.to_datetime(af.end_date)
af.created_at = pd.to_datetime(af.created_at)
af.updated_at = pd.to_datetime(af.updated_at)

In [11]:
# verify
af.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   path        900222 non-null  object        
 1   user_id     900223 non-null  int64         
 2   cohort_id   847330 non-null  float64       
 3   ip          900223 non-null  object        
 4   name        847330 non-null  object        
 5   start_date  847330 non-null  datetime64[ns]
 6   end_date    847330 non-null  datetime64[ns]
 7   created_at  847330 non-null  datetime64[ns]
 8   updated_at  847330 non-null  datetime64[ns]
 9   program_id  847330 non-null  float64       
dtypes: datetime64[ns](4), float64(2), int64(1), object(3)
memory usage: 75.5+ MB


In [12]:
# verify
af.head()

,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id
date_time,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0


In [13]:
# cache this frame too, but be sure to re-convert all dates/ times columns to datetimes after loading through
# pandas read_csv function
af.to_csv('reminder_convert_datetimes.csv')

### Exercise I 
Label students by the program they are in.

In [14]:
# view different programs
af.program_id.value_counts()

2.0    713365
3.0    103412
1.0     30548
4.0         5
Name: program_id, dtype: int64

In [15]:
# create columns that label students by the program they are in
af['front_end'] = af.program_id == 4
af['data_science'] = af.program_id == 3
af['java'] = af.program_id == 2
af['php'] = af.program_id == 1

In [16]:
# mira
af.head()

,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id,front_end,data_science,java,php
date_time,,,,,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,False,False,False,True
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,False,False,False,True
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,False,False,False,True
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,False,False,False,True
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,False,False,True,False


In [17]:
# create column called program that declares what program the student is in
af['program'] = af.program_id.map({1.0: 'Full-Stack PHP',
                                   2.0: 'Full-Stack Java',
                                   3.0: 'Data Science',
                                   4.0: 'Front-End'
                                  }
                                 )

In [18]:
# set a list of columns to map
programs = ['front_end', 'data_science', 'java', 'php']

# commence loop through list to map one-hot encoding for practical purposes like the .sum() function
for program in programs:
    af[program] = af[program].map({False: 0,
                                   True: 1
                                  }
                                 )

In [19]:
# alright what's going on here
af.head()

,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id,front_end,data_science,java,php,program
date_time,,,,,,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,0,0,0,1,Full-Stack PHP
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,0,0,0,1,Full-Stack PHP
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,0,0,0,1,Full-Stack PHP
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,0,0,0,1,Full-Stack PHP
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,0,0,1,0,Full-Stack Java


You did it! Too much work, mason. Too much work for question 1. All you needed was cell 17.

### Exercise II 
Is it possible to identify the user_id of a staff member?

Yes. Yes it is.

### Exercise III 
Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.

### Exercise IV 
Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2.

### Exercise V
Plot the access activity of these students.

### BONUS: Identify users who are viewing both the web dev and data science curriculum